In [1]:
from worldWeatherApi import WorldWeatherApi

In [2]:
weatherApi = WorldWeatherApi()

In [ ]:
# df = weatherApi.test_download(city="sao+paulo", start_date="2025-07-14", end_date="2025-07-14", freq=1)

Processando clima Cidade: sao+paulo


Retrieving weather data for sao+paulo


Currently retrieving data for sao+paulo: from 2025-07-14 to 2025-07-14


C:\Users\johns\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\wwo_hist\__init__.py:69: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  list_mon_begin = pd.date_range(start_date, end_date, freq='MS', closed='right')
C:\Users\johns\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\wwo_hist\__init__.py:74: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  list_mon_end = pd.date_range(start_date, end_date, freq='M', closed='left')


Time elapsed (hh:mm:ss.ms) 0:00:00.455994


export sao+paulo completed!




In [ ]:
import os
import pandas as pd

def load_weather_data(folder_path):
    # Lista para armazenar os dataframes de cada cidade
    dfs = []
    
    # Iterar sobre todos os arquivos CSV na pasta
    for filename in os.listdir(folder_path):
        print(filename)
        if filename.endswith(".csv"):
            # Criar o caminho completo do arquivo
            file_path = os.path.join(folder_path, filename)
            
            # Carregar o arquivo CSV em um dataframe
            df = pd.read_csv(file_path)
            
            # Extrair o nome do município (antes da vírgula e 'Portugal')
            city_name = filename.split(',')[0]  # Pega tudo antes da vírgula

            # Substituir "+" por espaço (" ") no nome do município
            city_name = city_name.replace("+", " ")
            
            # Adicionar o nome da cidade como uma coluna
            df['municipio'] = city_name
            
            # Adicionar o dataframe à lista
            dfs.append(df)
    
    # Concatenar todos os dataframes em um único dataframe
    combined_df = pd.concat(dfs, ignore_index=True)
    
    return combined_df


In [83]:
import pandas as pd

def generate_city_ranking(df, aggregation='mean', temp_min=20, temp_max=25, humidity_min=40, humidity_max=60, uv_max=5, precip_max=1, wind_speed_max=20):
    """
    Função para gerar o ranking de clima agradável por cidade, calculando a média ou mediana das variáveis,
    com pesos ajustados para cada variável.
    
    Parâmetros:
    - df (DataFrame): Dados climáticos diários.
    - aggregation (str): Tipo de agregação ('mean' ou 'median').
    - temp_min (int): Temperatura mínima aceitável para ser considerada agradável (Celsius).
    - temp_max (int): Temperatura máxima aceitável para ser considerada agradável (Celsius).
    - humidity_min (int): Umidade mínima aceitável.
    - humidity_max (int): Umidade máxima aceitável.
    - uv_max (int): Índice UV máximo aceitável.
    - precip_max (float): Precipitação máxima aceitável (em mm).
    - wind_speed_max (int): Velocidade máxima do vento aceitável (em km/h).
    
    Retorna:
    - DataFrame com as pontuações de clima agradável para cada cidade.
    """
    
    # Verificar se a coluna 'municipio' está presente no DataFrame
    if 'municipio' not in df.columns:
        raise ValueError("A coluna 'municipio' não foi encontrada no DataFrame.")
    
    # Agrupar por cidade e calcular a média ou mediana
    if aggregation == 'mean':
        df_agg = df.groupby('municipio').mean()
    elif aggregation == 'median':
        df_agg = df.groupby('municipio').median()
    else:
        raise ValueError("O parâmetro 'aggregation' deve ser 'mean' ou 'median'.")
    
    # Restaurar 'municipio' como coluna após o agrupamento
    df_agg['municipio'] = df_agg.index
    
    # Verificar se as colunas essenciais estão presentes após o agrupamento
    required_columns = ['maxtempC', 'mintempC', 'humidity', 'uvIndex', 'precipMM', 'windspeedKmph', 'FeelsLikeC']
    missing_columns = [col for col in required_columns if col not in df_agg.columns]
    if missing_columns:
        raise ValueError(f"As seguintes colunas estão faltando no DataFrame após o agrupamento: {', '.join(missing_columns)}")
    
    # Função para calcular a pontuação de cada variável
    def score_temperature(temp):
        if temp < temp_min:
            return 0
        elif temp > temp_max:
            return 0
        else:
            return 10  # Pontuação máxima para temperatura dentro da faixa agradável

    def score_humidity(humidity):
        if humidity < humidity_min or humidity > humidity_max:
            return 0
        else:
            return 10  # Pontuação máxima para umidade dentro da faixa agradável

    def score_uv(uv):
        if uv <= uv_max:
            return 10
        else:
            return 0

    def score_precipitation(precip):
        if precip <= precip_max:
            return 10
        else:
            return 0

    def score_wind_speed(wind_speed):
        if wind_speed <= wind_speed_max:
            return 10
        else:
            return 0

    def score_feels_like(feels_like, temp):
        if abs(feels_like - temp) <= 2:  # Se a sensação térmica estiver muito próxima da temperatura real
            return 10
        else:
            return 0
    
    # Aplicar as funções de pontuação para cada cidade
    df_agg['temp_score'] = df_agg['maxtempC'].apply(score_temperature)
    df_agg['humidity_score'] = df_agg['humidity'].apply(score_humidity)
    df_agg['uv_score'] = df_agg['uvIndex'].apply(score_uv)
    df_agg['precip_score'] = df_agg['precipMM'].apply(score_precipitation)
    df_agg['wind_score'] = df_agg['windspeedKmph'].apply(score_wind_speed)
    df_agg['feels_like_score'] = df_agg.apply(lambda row: score_feels_like(row['FeelsLikeC'], row['maxtempC']), axis=1)
    
    # Definir os pesos para cada variável
    temp_weight = 0.6
    feels_like_weight = 0.2
    other_weight = 0.2 / 4  # O restante será dividido entre as 4 outras variáveis
    
    # Calcular a pontuação total com os pesos ajustados
    df_agg['total_score'] = (
        (df_agg['temp_score'] * temp_weight) + 
        (df_agg['feels_like_score'] * feels_like_weight) + 
        (df_agg['humidity_score'] * other_weight) + 
        (df_agg['uv_score'] * other_weight) + 
        (df_agg['precip_score'] * other_weight) + 
        (df_agg['wind_score'] * other_weight)
    )
    
    # Normalizar a pontuação para que o total máximo seja 100
    df_agg['normalized_score'] = df_agg['total_score'] * 100  # Já está em uma escala de 0 a 100
    
    # Ordenar as cidades pelo ranking
    df_sorted = df_agg.sort_values(by='normalized_score', ascending=False)
    
    # Exibir as cidades com o clima mais agradável no topo
    return df_sorted[['municipio', 'maxtempC', 'mintempC', 'humidity', 'precipMM', 'windspeedKmph','temp_score', 'feels_like_score', 'humidity_score', 'uv_score', 'precip_score', 'wind_score', 'total_score', 'normalized_score']]

# Exemplo de uso:
# df_ranking = generate_city_ranking(df, aggregation='mean')
# ace_tools.display_dataframe_to_user(name="Ranking de Clima Agradável por Cidade", dataframe=df_ranking)


In [84]:
filePathClima = '..\\..\\Bases\\Clima\\worldWeatherApi\\'
df = load_weather_data(filePathClima)

Abrantes,Portugal.csv
Agueda,Portugal.csv
Aguiar+da+Beira,Portugal.csv
Alandroal,Portugal.csv
Albergaria-a-Velha,Portugal.csv
Albufeira,Portugal.csv
Alcacer+do+Sal,Portugal.csv
Alcanena,Portugal.csv
Alcobaca,Portugal.csv
Alcochete,Portugal.csv
Alcoutim,Portugal.csv
Alenquer,Portugal.csv
Alfandega+da+Fe,Portugal.csv
Alijo,Portugal.csv
Aljezur,Portugal.csv
Aljustrel,Portugal.csv
Almada,Portugal.csv
Almeida,Portugal.csv
Almeirim,Portugal.csv
Almodovar,Portugal.csv
Alpiarca,Portugal.csv
Alter+do+Chao,Portugal.csv
Alvaiazere,Portugal.csv
Alvito,Portugal.csv
Amadora,Portugal.csv
Amarante,Portugal.csv
Amares,Portugal.csv
Anadia,Portugal.csv
Angra+do+Heroismo,Portugal.csv
Ansiao,Portugal.csv
Arcos+de+Valdevez,Portugal.csv
Arganil,Portugal.csv
Armamar,Portugal.csv
Arouca,Portugal.csv
Arraiolos,Portugal.csv
Arronches,Portugal.csv
Arruda+dos+Vinhos,Portugal.csv
Aveiro,Portugal.csv
Avis,Portugal.csv
Azambuja,Portugal.csv
Baiao,Portugal.csv
Barcelos,Portugal.csv
Barrancos,Portugal.csv
Barreiro,Port

In [85]:
df

,date_time,maxtempC,mintempC,totalSnow_cm,sunHour,uvIndex,moon_illumination,moonrise,moonset,sunrise,...,cloudcover,humidity,precipMM,pressure,tempC,visibility,winddirDegree,windspeedKmph,location,municipio
0,2024-06-30,24,15,0.0,9.0,4,27,01:51 AM,03:37 PM,06:09 AM,...,55,67,0.7,1016,24,10,305,14,"Abrantes,Portugal",Abrantes
1,2024-07-01,30,13,0.0,15.0,6,26,02:18 AM,04:50 PM,06:09 AM,...,30,65,0.0,1019,30,7,320,11,"Abrantes,Portugal",Abrantes
2,2024-07-02,35,17,0.0,15.0,7,19,02:50 AM,06:04 PM,06:10 AM,...,3,50,0.0,1016,35,10,234,11,"Abrantes,Portugal",Abrantes
3,2024-07-03,34,17,0.0,15.0,6,12,03:29 AM,07:15 PM,06:10 AM,...,0,51,0.0,1013,34,10,313,12,"Abrantes,Portugal",Abrantes
4,2024-07-04,38,16,0.0,15.0,7,5,04:16 AM,08:20 PM,06:11 AM,...,3,47,0.0,1011,38,10,273,9,"Abrantes,Portugal",Abrantes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113094,2025-06-26,29,13,0.0,15.0,5,0,06:41 AM,10:37 PM,06:03 AM,...,3,67,0.0,1021,29,8,187,7,"Vouzela,Portugal",Vouzela
113095,2025-06-27,34,17,0.0,15.0,6,7,07:56 AM,11:15 PM,06:03 AM,...,0,58,0.0,1022,34,10,172,6,"Vouzela,Portugal",Vouzela
113096,2025-06-28,37,20,0.0,15.0,7,15,09:10 AM,11:45 PM,06:04 AM,...,11,48,0.0,1021,37,10,188,6,"Vouzela,Portugal",Vouzela
113097,2025-06-29,42,23,0.0,14.0,7,22,10:20 AM,No moonset,06:04 AM,...,25,36,0.1,1019,42,10,95,7,"Vouzela,Portugal",Vouzela


In [86]:
# Exemplo de uso:
df_ranking = generate_city_ranking(df)


C:\Users\johns\AppData\Local\Temp\ipykernel_24048\3262375968.py:29: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_agg = df.groupby('municipio').mean()


In [87]:
df_ranking

,municipio,maxtempC,mintempC,humidity,precipMM,windspeedKmph,temp_score,feels_like_score,humidity_score,uv_score,precip_score,wind_score,total_score,normalized_score
municipio,,,,,,,,,,,,,,
Porto Santo,Porto Santo,20.092896,19.161202,75.027322,0.944262,23.707650,10,10,0,10,10,0,9.0,900.0
Abrantes,Abrantes,23.240437,11.439891,69.928962,2.772404,11.592896,10,0,0,10,0,10,7.0,700.0
Trancoso,Trancoso,20.448087,10.224044,72.087432,4.241803,7.609290,10,0,0,10,0,10,7.0,700.0
Marinha Grande,Marinha Grande,20.538251,11.836066,77.245902,2.758197,12.218579,10,0,0,10,0,10,7.0,700.0
Marvao,Marvao,20.991803,11.560109,75.647541,3.365027,10.898907,10,0,0,10,0,10,7.0,700.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Miranda do Douro,Miranda do Douro,19.494536,8.868852,65.538251,2.001093,10.450820,0,0,0,10,0,10,1.0,100.0
Melgaco,Melgaco,18.655738,11.275956,77.617486,5.042077,10.387978,0,0,0,10,0,10,1.0,100.0
Meda,Meda,18.090164,8.240437,70.341530,2.374044,10.819672,0,0,0,10,0,10,1.0,100.0


In [88]:
# export df
filePathClimaRaking = filePathClima + '\\ranking\\' + 'ClimaRaking' + '.csv'
df_ranking.to_csv(filePathClimaRaking, sep=',', encoding='utf-8', doublequote=True, decimal=',')

In [ ]:
def show_sorted_weather(df, score_type="normalized_score"):
    """
    Função para mostrar o clima de forma ordenada, classificando pela pontuação total ou normalizada.
    
    Parâmetros:
    - df (DataFrame): Dados climáticos com a coluna de pontuação.
    - score_type (str): Tipo de pontuação para ordenação. Pode ser "total_score" ou "normalized_score".
    
    Retorna:
    - DataFrame ordenado com base no tipo de pontuação escolhido.
    """
    # Verificar se a coluna de pontuação escolhida existe no DataFrame
    if score_type not in df.columns:
        raise ValueError(f"A coluna '{score_type}' não foi encontrada no DataFrame.")
    
    # Verificar se as colunas essenciais estão presentes no DataFrame
    required_columns = ['municipio', 'maxtempC', 'mintempC', 'humidity', 'precipMM', 'windspeedKmph']
    missing_columns = [col for col in required_columns if col not in df.columns]
    if missing_columns:
        raise ValueError(f"As seguintes colunas estão faltando no DataFrame: {', '.join(missing_columns)}")
    
    # Ordenar o DataFrame com base no score escolhido (de forma decrescente)
    sorted_df = df.sort_values(by=score_type, ascending=False)
    
    # Exibir as cidades com o clima mais agradável no topo
    return sorted_df[['municipio', score_type, 'maxtempC', 'mintempC', 'humidity', 'precipMM', 'windspeedKmph']]

# Exemplo de uso:
# df_sorted = show_sorted_weather(df, score_type="normalized_score")
# ace_tools.display_dataframe_to_user(name="Ranking de Clima Agradável", dataframe=df_sorted)


In [ ]:
df

In [89]:
show_sorted_weather(df_ranking)

,municipio,normalized_score,maxtempC,mintempC,humidity,precipMM,windspeedKmph
municipio,,,,,,,
Porto Santo,Porto Santo,900.0,20.092896,19.161202,75.027322,0.944262,23.707650
Baiao,Baiao,700.0,21.224044,12.377049,70.437158,2.532240,11.385246
Vila Real de Santo Antonio,Vila Real de Santo Antonio,700.0,22.098361,14.789617,67.590164,2.066393,14.300546
Alcanena,Alcanena,700.0,22.896175,11.715847,71.961749,2.861475,11.653005
Alcacer do Sal,Alcacer do Sal,700.0,22.016393,13.434426,71.639344,2.508197,14.868852
...,...,...,...,...,...,...,...
Moncao,Moncao,100.0,18.560109,12.377049,77.491803,4.236339,14.743169
Sintra,Sintra,100.0,19.732240,13.366120,75.308743,2.957377,17.827869
Seixal,Seixal,100.0,19.590164,13.092896,76.486339,2.766120,17.568306
